### Modeling of Cluster Distributions

The objectives of this demo are to 1) introduce the four goals that we would like to achieve by developing this cluster counting module, 2) explain the algorithms for each goal, and 3) demonstrate the procedures to achieve each goal with examples. This report includes five parts as below.

1. Introduction
2. Goal One - Identify Distinct Clusters (understand the lattice/structure)
3. Goal Two - Count Clusters
4. Goal Three - Generate Random Structures With/Without Rules
5. Goal Four - Titrate Clusters

### Introduction

The reactivity of meterials always depends on the distribution of important groups of atoms. Here we will refer the important groups of atoms as clusters. Given the bulk material property, such as the element ratio, we want to compute the distributions of the clusters in order to quantitatively understand the reactivity. The ultimate goal of this cluster counting module is to statistically compute the cluster distributions for different conditions, such as different crystal structures, different rules for atom locations, and different counting and titration priorities. This ultimate goal can be further divided into four specific goals. First, given a crystal structure, we want to understand the lattice and be able to identify distinct clusters. Next, we want to be able to count different type of clusters given a structure configuration (which specifies the atom type at each site). Thirdly, we would like to generate randome structure configurations for any given element ratio. Last, we want to titrate clusters one by one to avoid double counting clusters that share sites with each other. 

### Goal One - Identify Distinct Clusters (understand the lattice/structure)

Algorithm 

To understand the distribution of clusters, we first need to distinguish between different clusters. Clusters differ from each other based on atom distances and symmetries. 

To achieve this goal, we took advantage of the Alloy Theoretic Automated Toolkit (ATAT) developed by Axel van de Walle. In ATAT, the corrdump program takes lattice parameters and site positions as input, determines the space group of the lattice, and find all symmetrically distinct clusters based on the space group. When analyzing distinct clusters, corrdump only count for sites which are possbile for at least two types of elements. The sites which can accommodate only one type of element will only help with analyzing symmetries. The installation and modification of ATAT can be found in supporting information.

Input:
1. lat.in  
2. str_dim.txt

Procedures:
1. Run the two python files (classes.py and utilities.py) which contain the useful functions in jupyter notebook (or import the them in a python file).
        %run classes.py
        %run utilities.py
        (from classes import * )
        (from utilities import *)
2. Initialize a class of Lattice with lat.in: 
        Lattice(folder_path for lat.in)
3. Run corrdump program to generate clusters.out file containing the information for each cluster type: 
        corrdump -l=[lat.in file path] -cf=[clusters.out file path] -2=[max distance for 2-body cluster] -3=[max distance for 3-body cluster]
4. Visualize the cluster example given by corrdump for each type: 
        lattice.read_clusters_out()
        lattice.visualize_cluster(cluster_type)
5. Initialize a class of Structure with lattice parameters and structure dimensions, and prepare str.out: 
        Structure(lattice, folder_path for lat.in and str_dim.txt)
        structure.prepare_str_out()
6. Run corrdump program to generate a full list of clusters for a super cell defined by the structure dimensions:
        corrdump -l=[lat.in file path] -s=[str.out file path] -cf=[clusters.out file path] -2=[max distance for 2-body cluster] -3=[max distance for 3-body cluster] >> [cluster_list.csv file path]
7. Read the full cluster list and visulaize clusters for each type:
        structure.read_cluster_list()
        structure.visualize_one_cluster_one_example()

Output:
1. A lattice class which contains:  
1) lattice parameters: a, b, c, alpha, beta, gamma  
2) lattice constants: u, v, w  
3) sites information: site index, atom types, xyz coordinates, fractional coordinates  
4) cluster types: number of atoms in the cluster, maximum distance between two atoms in the cluster, multiplicity in one lattice unit cell, one example cluster represented in fractional coordinates for each type.  
        
2. A structure class which contains:   
1) lattice parameters: a, b, c, alpha, beta, gamma  
2) lattice constants: u, v, w  
3) structure constants: nu, nv, nw  
4) sites information stored in a dataframe: site index, atom types, xyz coordinates, fractional coordinates  
5) cluster types: all clusters represented in fractional coordinates, in xyz coordinates in site indices for each type in one super cell.  

Example:

In [1]:
#import other useful packages
import pandas as pd
import numpy as np
import os
from ase import Atoms
from ase.io import read, write
from ase.visualize import view

In [2]:
#run the two python files
%run classes.py
%run utilities.py

In [4]:
#prepare lat.in and str_dim.txt for simple cube and put them in the folder called simple_cube 
folder_path = 'simple_cube'
#initialize a class of Lattice with lat.in:
lattice = Lattice(folder_path)

In [15]:
#set the maxmum distances between 2 atoms in 2-body clusters and that in 3-body clusters
maxdis_2 = 15
maxdis_3 = 15
#run corrdump to generate clusters.out file in terminal:
#the folder path for lat.in and clusters.out has been specified before
#the return of this line of code is either 0 or 256: 0 means no error message, and 256 means there is at least one error message; you can see the error messages in the terminal; if there is no str.out file, it should return 256 and there should be an error message (Unable to open structure file) in terminal; that's fine.
os.system('corrdump -l={0}/lat.in -cf={0}/clusters.out -2={1} -3={2}'.format(folder_path, maxdis_2, maxdis_3))

256

In [16]:
#read clusters.out
lattice.read_clusters_out()

In [9]:
#visualize the cluster example given by corrdump for one type
cluster_type='2-5'
lattice.visualize_cluster(cluster_type)
c= read(folder_path+'/lattice_clusters/xyzs/cluster-{}.xyz'.format(cluster_type))
view(c)

In [10]:
#initialize a class of Structure with lattice parameters and structure dimensions
structure = Structure(lattice=lattice, folder_path=folder_path)
structure.prepare_str_out()

In [13]:
#run corrdump program in terminal to generate a full list of clusters for a super cell defined by the structure dimensions; again, the return of this line of code is either 0 or 256: 0 means no error message, and 256 means there is at least one error message; you can see the error messages in the terminal

os.system('corrdump -l={0}/lat.in -s={0}/str.out -cf={0}/clusters.out -2={1} -3={2} >> {0}/cluster_list.csv'.format(folder_path, maxdis_2, maxdis_3))

0

In [17]:
#read the full cluster list and visulaize clusters for each type
structure.read_cluster_list()

In [18]:
#create xyz and image files for clusters in a specific type
cluster_type='3-3'
structure.visualize_one_cluster_type_all_examples(cluster_type)

In [19]:
#visualize one cluster example
cluster_example='3-3-1'
c= read(folder_path+'/structure_clusters_rep/xyzs/cluster-{}.xyz'.format(cluster_example))
view(c)
c= read(folder_path+'/structure_clusters_no_rep/xyzs/cluster-{}.xyz'.format(cluster_example))
view(c)

### Goal Two - Count Clusters

Algorithms 

The second goal is to count specific clusters for a given structure configuration. Usually for the clusters that we want to count, there are two requirements: 1) having centain element at each site, and 2) being certain cluster type, such as Al-Al pairs in 6-membered-rings. Other rules, such as excluding certain cluster types, might also exist. For example, when there are 3 Al atoms not adjacent with each other in a 6-membered-ring, we may not want to count any Al-Al pair in this 6-ring. The codes to complete this task contain 2 parts.

1. If there is any excluding cluster type, the first step is to create a set of all the clusters that we want to exclude in one super cell. We can go through the cluster list for each excluding type, add every cluster as well as its subclusters into the excluding cluster set. For example, 3Al in 6-membered-rings is a particular 3-body cluster type. We will add each 3-body cluster in this type, as well as each 2-body cluster and 1-body cluster in this 3-body cluster, to the excluding set.  

2. Then we will go through the cluster list for each of the cluster type that we are interested in. For every cluster in this type, if it has exactly the required element at each site(given by the structure configuration), and it doesn't belong to excluding cluster set, we will add the count for its cluster type by 1.

Input:
1. structure vector
2. counting types and excluding types

Procedures:
1. count clusters for one structure vector:  
   structure.count_clusters_str_config(str_vec, counting_types, excluding_types)  
2. or count clusters for multiple structure vectors:   
structure.count_clusters_multi_configs(str_vecs, counting_types, excluding_types)

Output:
1. counting_results:  
    number of clusters within one super cell for each cluster type in counting types

Example:

In [11]:
#initialize a structure vector with 1 at all sites with multiple atom types and 0 at all sites with only one atom type
str_vec=[1 if (structure.sites.iloc[i]['multi_atoms']==True) else 0 for i in range(len(structure.sites.index)) ]

In [12]:
#count clusters without excluding types
structure.count_clusters_str_config(str_vec, counting_types=['1-1','2-1','2-2'])

defaultdict(int, {'1-1': 12, '2-1': 24, '2-2': 24})

In [13]:
#count clusters with excluding types
structure.count_clusters_str_config(str_vec, counting_types=['1-1','2-1','2-2'], excluding_types=['3-2'])

defaultdict(int, {'1-1': 0, '2-1': 0, '2-2': 24})

### Goal Three - Randomly generate structure vectors with/without rules

Algorithms:   
When researchers design functional materials, the element ratio is usually the first property to control. Different element ratios will result in different cluster distributions. Moreover, for a given element ratio, the cluster distribution may vary with the distribution of elements within the structure. Thus the third goal here is to randomly generate numbers of structure configurations for a given element ratio to statistically analyze the cluster distributions. When structure configuration is generated, certain rules may apply. For example, Löwenstein's rule requires no first nearest neighbors for Al. Another rule is the probability of centain atoms at different sites may not be the same. For ferrierite, there are four types of T-sites. The probability of Al at different T-sites may vary with the synthesis process. To generate random configurations with/without rules, we need 3 steps.

1. Randomly initialize a structure configuration(structure vector) which has the required atom ratios and the required site probability.  
2. If there are penalized cluster types, create a penalty dictionary. Give a penalty factor for each penalized cluster type. Then the overall penalty for a structure configuration is the summation of the number of penalized clusters times the penalty factor for all penalized types.
3. Randomly swap different atoms at different sites. If different site types has different probabilities, only swap sites with the same type. If the swap results in a not greater overall panalty, swap the atoms; othewise, the swap probability follows the Boltzmaan function of the penalty difference. Keep swapping until the overall penalty becomes stable.


Input:
1. atom ratio
2. rules: penalty dictionary containing the penalty factors for all the penalized cluster types

Procedures:
1. random_config_swap(self, atom_num, penalty={}, prob={}, num_vecs=1, num_step=100, burn_in_period=10, vis=0, ptfile='')

Output:
1. a set of structure vectors that meet the requirements of atom ratio and site probability as well as minimize the total penalty.

Example:

In [14]:
#no 1NN for Al and only one type of site
prob={'1':1}
penalty={'2-1':10}
Al_ratio=0.4
Al_num=int(Al_ratio*len(structure.sites[structure.sites.multi_atoms==True].index))
str_vecs=structure.random_config_swap(Al_num, penalty=penalty, prob=prob,num_vecs=2, num_step=100,vis=1, process=1, ptfile=1)

In [15]:
str_vecs

[[0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0]]

In [16]:
c= read(folder_path+'/random_config_process/xyzs/swap-1-final.xyz')
view(c)

In [17]:
c= read(folder_path+'/random_config_process/xyzs/swap-2-final.xyz')
view(c)

### Goal Four - Titrate Clusters

Algorithms 

The ultimate goal of the cluster distribution analysis is to understand the reaction activity with the material structure. When counting the clusters, we count both clusters if they both meet the requirements when they share sites with each other. While in reactions, if one cluster reacts, the atoms in the cluster will participate in the reaction. The clusters that share site with the reacted cluster cannot further react. So we need a titration function to titrate the clusters one by one. After each titration, we mark the sites in the titrated cluster as used and won't titrate clusters sharing sites with the titrated cluster in the future. Another thing that we need to consider is that, not all the clusters types have the same reaction priority. Some types may react first and only all the clusters in these types are all used up, the other types of clusters can start to react. To complete this task, we need 3 steps.

1. If there is any excluding cluster type, create an excluding cluster list as in the counting function. Go through the cluster list for each excluding type, add every cluster as well as its subclusters into the excluding cluster set.  
2. Create a titration group for the cluster types with the highest priority. Create an available cluster list with all the clusters that have the required atoms, being the type in the titration group, and not in the excluding cluster set. Then we start to titrate the clusters one by one. After each titration, add 1 for the cluster type that the titrated cluster belongs to, change the sites in the titrated cluster to used sites in the structure configuration, and remove all the clusters that share sites with it from the available cluster list. Keep titrating until no cluster is in the available list. 
3. Create another titration group for the cluster types with the second highest priority. Create an available cluster list as in step 2 with the structure configuration that already mark all the used sites. Titrate as in step 2. 

Input:
1. structure vector
2. titrating groups:[[titration types with the highest priority],[titration types with the second highest priority]...]
3. excluding types

Procedures:
1. titrate_clusters_multi_configs(self, str_vecs, titration_groups, excluding_types, titrate_num)

Output:
1. titration_results:  
    number of clusters within one super cell for each cluster type in titration groups

Example:

In [18]:
#generate 5 structure configurations with no rules
Al_ratio=0.4
Al_num=int(Al_ratio*len(structure.sites[structure.sites.multi_atoms==True].index))
str_vecs=structure.random_config_swap(Al_num, num_vecs=5, num_step=100)

In [19]:
titration_groups=[['2-1','2-2','2-3'],['1-1']]
excluding_types=[]

In [20]:
#titrate multiple configurations for 10 times and keep the mean values of the tiration results
structure.titrate_clusters_multi_configs(str_vecs, titration_groups= titration_groups,titrate_num=10,hist=0)

{'1-1': [[0.2], [0.8], [0.0], [0.8], [1.0]],
 '2-1': [[0.8], [0.3], [1.4], [0.6], [1.0]],
 '2-2': [[0.5], [0.8], [0.4], [0.6], [0.4]],
 '2-3': [[0.6], [0.5], [0.2], [0.4], [0.1]]}

In [21]:
#titrate multiple configurations for 10 times and keep the all the titration results
structure.titrate_clusters_multi_configs(str_vecs, titration_groups=titration_groups,titrate_num=10,hist=1)

{'1-1': [[2, 0, 0, 0, 0, 0, 0, 2, 0, 0],
  [0, 0, 2, 0, 0, 0, 2, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [2, 0, 0, 0, 0, 2, 0, 0, 0, 0],
  [0, 0, 0, 2, 0, 0, 2, 0, 0, 0]],
 '2-1': [[1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [1, 0, 1, 0, 1, 1, 0, 1, 1, 1],
  [1, 2, 2, 0, 1, 1, 1, 1, 2, 0],
  [0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
  [2, 0, 0, 0, 2, 2, 1, 0, 2, 2]],
 '2-2': [[0, 1, 1, 1, 1, 1, 1, 0, 1, 1],
  [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
  [0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
  [0, 2, 2, 0, 0, 0, 0, 2, 0, 0]],
 '2-3': [[0, 1, 1, 1, 1, 1, 1, 0, 1, 1],
  [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 1, 1, 1, 1, 0, 2],
  [0, 1, 1, 0, 0, 0, 1, 1, 1, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]}